In [1]:
# ===============================================================
# 00_setup.ipynb - Configuración Inicial del Proyecto
# ===============================================================
#
# **Propósito**: 
# - Validar estructura del proyecto
# - Verificar dependencias instaladas
# - Configurar paths usando utils_shared.py
# - Preparar entorno para ejecutar notebooks 01_ y 02_
#
# **Cuándo ejecutar**:
# - Primera vez que clonas el repositorio
# - Después de cambios en la estructura de carpetas
# - Para verificar que todo está correctamente instalado
#
# **Qué NO hace**:
# - No procesa datos (eso es 01_eda_understanding.ipynb)
# - No crea splits (eso es 02_create_splits.ipynb)
# - No entrena modelos (eso es 02_baseline_*.ipynb)
#
# ===============================================================

import os, sys, pathlib, yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------------------------------------------
# 1. Detección de entorno (Local vs Google Colab)
# ---------------------------------------------------------------
try:
    import google.colab
    IN_COLAB = True
    print("📦 Entorno: Google Colab")
except ImportError:
    IN_COLAB = False
    print("📦 Entorno: Local (Jupyter/VSCode)")

if IN_COLAB:
    print("   💡 En Colab puedes montar Google Drive para persistir datos")
else:
    print("   💡 Ejecutando en entorno local")

📦 Entorno: Local (Jupyter/VSCode)
   💡 Ejecutando en entorno local


In [2]:
# ===============================================================
# 2. Configuración de Paths usando utils_shared.py
# ===============================================================
#
# Intentamos usar utils_shared.py para centralizar configuración.
# Si no está disponible, usamos configuración manual.
#
# ===============================================================

try:
    from utils_shared import setup_paths
    print("✅ Cargando paths desde utils_shared.py")
    
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    FORK_PATH = paths['FORK_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    FIGS_PATH = paths['FIGS_PATH']
    
    print("   Usando configuración centralizada")
    
except ImportError:
    print("⚠️ utils_shared.py no encontrado, usando configuración manual")
    
    BASE_PATH = pathlib.Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    FORK_PATH = BASE_PATH / "Spanish_Psych_Phenotyping_PY"
    SPLITS_PATH = DATA_PATH / "splits"
    FIGS_PATH = DATA_PATH / "figs"
    
    # Crear directorios si no existen
    DATA_PATH.mkdir(exist_ok=True)
    SPLITS_PATH.mkdir(exist_ok=True)
    FIGS_PATH.mkdir(exist_ok=True)

# En Colab → permitir montaje de Google Drive
if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        DRIVE_DATA_PATH = pathlib.Path("/content/drive/MyDrive/psych-data")
        
        if DRIVE_DATA_PATH.exists():
            DATA_PATH = DRIVE_DATA_PATH
            SPLITS_PATH = DATA_PATH / "splits"
            FIGS_PATH = DATA_PATH / "figs"
            print("📁 Usando datos desde Google Drive")
    except Exception as e:
        print(f"⚠️ No se pudo montar Google Drive: {e}")

print(f"\n📁 Paths configurados:")
print(f"   BASE_PATH:   {BASE_PATH}")
print(f"   DATA_PATH:   {DATA_PATH}")
print(f"   FORK_PATH:   {FORK_PATH}")
print(f"   SPLITS_PATH: {SPLITS_PATH}")
print(f"   FIGS_PATH:   {FIGS_PATH}")

✅ Cargando paths desde utils_shared.py
   Usando configuración centralizada

📁 Paths configurados:
   BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
   DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
   FORK_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY
   SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits
   FIGS_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/figs


In [3]:
# ===============================================================
# 3. Verificación de Estructura del Proyecto
# ===============================================================
#
# Verifica que existen las carpetas clave del proyecto.
# Si faltan, las crea automáticamente (excepto FORK_PATH).
#
# ===============================================================

print("\n🔍 Verificando estructura del proyecto...\n")

# Directorios que DEBEN existir (se crean si faltan)
auto_create_dirs = {
    'data/': DATA_PATH,
    'data/splits/': SPLITS_PATH,
    'data/figs/': FIGS_PATH,
    'notebooks/': BASE_PATH / "notebooks",
    'configs/': BASE_PATH / "configs"
}

# Directorios que NO se crean automáticamente (requieren acción del usuario)
required_dirs = {
    'Spanish_Psych_Phenotyping_PY/': FORK_PATH
}

# Crear directorios faltantes
for name, path in auto_create_dirs.items():
    if path.exists():
        print(f"✅ {name:30} {path}")
    else:
        path.mkdir(parents=True, exist_ok=True)
        print(f"🆕 {name:30} {path} (creado)")

# Validar directorios que requieren acción manual
for name, path in required_dirs.items():
    if path.exists():
        print(f"✅ {name:30} {path}")
    else:
        print(f"❌ {name:30} {path} (NO ENCONTRADO)")
        print(f"   ⚠️ El fork es necesario para baseline rule-based")
        print(f"   💡 Clona desde: https://github.com/[owner]/Spanish_Psych_Phenotyping_PY")

print("\n" + "="*60)


🔍 Verificando estructura del proyecto...

✅ data/                          /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
✅ data/splits/                   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits
✅ data/figs/                     /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/figs
✅ notebooks/                     /Users/manuelnunez/Projects/psych-phenotyping-paraguay/notebooks
✅ configs/                       /Users/manuelnunez/Projects/psych-phenotyping-paraguay/configs
✅ Spanish_Psych_Phenotyping_PY/  /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY



In [4]:
# ===============================================================
# 4. Verificación de Dependencias de Python
# ===============================================================
#
# Verifica que las librerías necesarias están instaladas.
# Agrupa por categoría para diagnóstico más claro.
#
# ===============================================================

print("\n🔍 Verificando dependencias de Python...\n")

dependencies = {
    'Core Data Science': ['pandas', 'numpy', 'scipy'],
    'Machine Learning': ['sklearn', 'transformers', 'torch'],
    'Visualización': ['matplotlib', 'seaborn', 'plotly'],
    'NLP/Texto': ['nltk', 'spacy'],
    'Utilities': ['yaml', 'tqdm']
}

all_ok = True

for category, libs in dependencies.items():
    print(f"📦 {category}:")
    for lib in libs:
        try:
            __import__(lib)
            print(f"   ✅ {lib}")
        except ImportError:
            print(f"   ❌ {lib} (NO INSTALADO)")
            all_ok = False
    print()

if all_ok:
    print("✅ Todas las dependencias están instaladas correctamente")
else:
    print("⚠️ Faltan dependencias. Instalar con:")
    print("   pip install -r requirements.txt")

print("="*60)


🔍 Verificando dependencias de Python...

📦 Core Data Science:
   ✅ pandas
   ✅ numpy
   ✅ scipy

📦 Machine Learning:
   ✅ sklearn
   ✅ transformers
   ✅ torch

📦 Visualización:
   ✅ matplotlib
   ✅ seaborn
   ✅ plotly

📦 NLP/Texto:
   ✅ transformers
   ✅ torch

📦 Visualización:
   ✅ matplotlib
   ✅ seaborn
   ✅ plotly

📦 NLP/Texto:
   ✅ nltk
   ✅ nltk
   ✅ spacy

📦 Utilities:
   ✅ yaml
   ✅ tqdm

✅ Todas las dependencias están instaladas correctamente
   ✅ spacy

📦 Utilities:
   ✅ yaml
   ✅ tqdm

✅ Todas las dependencias están instaladas correctamente


In [5]:
# ===============================================================
# 5. Verificación de Archivos de Datos
# ===============================================================
#
# Verifica la disponibilidad de archivos clave en el flujo de trabajo:
# - ips_raw.csv (entrada original)
# - ips_clean.csv (salida de 01_eda)
# - splits/ (salida de 02_create_splits)
#
# ===============================================================

print("\n🔍 Verificando archivos de datos...\n")

# Archivos clave y su propósito
data_files = {
    'ips_raw.csv': {
        'path': DATA_PATH / 'ips_raw.csv',
        'required': True,
        'source': 'Archivo original del dataset',
        'used_by': '01_eda_understanding.ipynb'
    },
    'ips_clean.csv': {
        'path': DATA_PATH / 'ips_clean.csv',
        'required': False,
        'source': 'Generado por 01_eda_understanding.ipynb',
        'used_by': '02_create_splits.ipynb'
    },
    'splits/dataset_base.csv': {
        'path': SPLITS_PATH / 'dataset_base.csv',
        'required': False,
        'source': 'Generado por 02_create_splits.ipynb',
        'used_by': 'Todos los 02_baseline_*.ipynb'
    },
    'splits/train_indices.csv': {
        'path': SPLITS_PATH / 'train_indices.csv',
        'required': False,
        'source': 'Generado por 02_create_splits.ipynb',
        'used_by': 'Todos los 02_baseline_*.ipynb'
    },
    'splits/val_indices.csv': {
        'path': SPLITS_PATH / 'val_indices.csv',
        'required': False,
        'source': 'Generado por 02_create_splits.ipynb',
        'used_by': 'Todos los 02_baseline_*.ipynb'
    }
}

missing_critical = []

for name, info in data_files.items():
    exists = info['path'].exists()
    required = info['required']
    
    if exists:
        print(f"✅ {name:30}")
        print(f"   📄 {info['path']}")
        
        # Mostrar info adicional si es CSV
        if info['path'].suffix == '.csv':
            try:
                df_info = pd.read_csv(info['path'], nrows=1)
                print(f"   📋 Columnas: {list(df_info.columns)}")
            except:
                pass
    else:
        status = "❌ CRÍTICO" if required else "⚠️ Pendiente"
        print(f"{status} {name:30} (NO ENCONTRADO)")
        print(f"   💡 {info['source']}")
        print(f"   🔗 Usado por: {info['used_by']}")
        
        if required:
            missing_critical.append(name)
    
    print()

# Resumen final
print("="*60)
if missing_critical:
    print(f"❌ Faltan {len(missing_critical)} archivo(s) crítico(s):")
    for f in missing_critical:
        print(f"   - {f}")
    print("\n⚠️ No podrás ejecutar los notebooks hasta tener estos archivos")
else:
    print("✅ Archivos críticos disponibles")
    print("💡 Puedes ejecutar 01_eda_understanding.ipynb para comenzar")


🔍 Verificando archivos de datos...

✅ ips_raw.csv                   
   📄 /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_raw.csv
   📋 Columnas: ['Archivo', 'Prontuario', 'Nombre Paciente', 'Sexo', 'Fecha Nacimiento', 'N° Consulta', 'Id', 'Fecha Consulta', 'Motivo Consulta', 'Tipo']

✅ ips_clean.csv                 
   📄 /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_clean.csv
   📋 Columnas: ['id_paciente', 'fecha', 'etiqueta', 'texto']

✅ splits/dataset_base.csv       
   📄 /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/dataset_base.csv
   📋 Columnas: ['row_id', 'texto', 'etiqueta']

✅ splits/train_indices.csv      
   📄 /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/train_indices.csv
   📋 Columnas: ['row_id']

✅ splits/val_indices.csv        
   📄 /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/val_indices.csv
   📋 Columnas: ['row_id']

✅ Archivos críticos disponibles
💡 Puedes ejecutar 01_eda